In [338]:
import random
from random import choices
import pandas as pd
import numpy as np
from pathlib import Path

In [339]:
months_list = ["January","February","March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

In [340]:
# Load data
housing_df = pd.read_csv("median_rents_multi_indexed.csv")

airbnb_data = pd.read_csv("raw_data/Airbnb_Texas_Rentals.csv")
airbnb_data = airbnb_data.rename(columns={'bedrooms_count':'Bedrooms'})

housing_df.sort_values("Date",ascending=True)

,RegionName,Bedrooms,Date,Median_Rent
0,Austin,3,2010-03,1300.0
1,Austin,3,2010-04,1297.5
2,Austin,3,2010-05,1500.0
3,Austin,3,2010-06,1500.0
4,Austin,3,2010-07,1310.0
...,...,...,...,...
13922,McAllen,2,2019-12,855.0
13923,Abilene,2,2019-12,650.0
13924,Carrollton,2,2019-12,1505.0
13911,Brownsville,2,2019-12,787.5


In [341]:
columns = airbnb_data.columns
columns = columns[1:5]
airbnb_df = airbnb_data.loc[:,columns]
months_index = [f'0{months_list.index(month)+1}' for month in months_list]
months_index = [month_index[-2:] for month_index in months_index]
airbnb_df["date_of_listing"] = [date.replace(" ","") for date in airbnb_df["date_of_listing"]]
airbnb_df["year"] = [f"{date[-4:]}" for date in airbnb_df["date_of_listing"]]
airbnb_df["month"] = [f"{months_index[months_list.index(date[:-4])]}" for date in airbnb_df["date_of_listing"]]
#Citation: https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-pandas-dataframe
airbnb_df["Date"] = airbnb_df[['year','month']].agg('-'.join,axis=1)
airbnb_df = airbnb_df.drop(columns=["date_of_listing","year","month"])
airbnb_df["Num_ABNB_Listings"] = 0

In [342]:
airbnb_df

,average_rate_per_night,Bedrooms,city,Date,Num_ABNB_Listings
0,$27,2,Humble,2016-05,0
1,$149,4,San Antonio,2010-11,0
2,$59,1,Houston,2017-01,0
3,$60,1,Bryan,2016-02,0
4,$75,2,Fort Worth,2017-02,0
...,...,...,...,...,...
18254,$60,1,Dallas,2013-03,0
18255,$99,2,San Antonio,2015-06,0
18256,$13,1,Dallas,2016-12,0
18257,$65,2,San Antonio,2016-10,0


In [347]:
airbnb_df.rename(columns={'city':'RegionName'},inplace=True)
airbnb_df.sort_values("RegionName")
for value in airbnb_df["Bedrooms"]:
    try:
        airbnb_df["Bedrooms"] = int(value)

    except KeyError:
        airbnb_df.drop(airbnb_df[value])
    
airbnb_df.sort_values("Bedrooms")

,average_rate_per_night,Bedrooms,RegionName,Date,Num_ABNB_Listings
0,$27,3,Humble,2016-05,0
12164,$98,3,Houston,2017-01,0
12165,$402,3,Marble Falls,2017-03,0
12166,$395,3,Marble Falls,2016-01,0
12167,$375,3,Boerne,2015-09,0
...,...,...,...,...,...
6083,$300,3,Fort Worth,2013-01,0
6082,$36,3,San Antonio,2017-03,0
6081,$82,3,Frisco,2013-11,0
6087,$27,3,Houston,2016-06,0


In [404]:
def multi_index_data(df):
    arrays = [df["RegionName"],
            df["Bedrooms"]]

    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples, names=['RegionName', 'Bedrooms'])

    df = df.set_index(index)
    df = df.drop(columns=["RegionName","Bedrooms"])
    df = df.sort_index()
    return df

In [405]:
airbnb_df_new = multi_index_data(airbnb_df)
airbnb_df_new

average_rate_per_night     Date  Num_ABNB_Listings
RegionName Bedrooms                                                   
Abilene    3                          $500  2016-07                  0
           3                           $85  2015-12                  0
           3                           NaN  2015-03                  0
           3                           $85  2013-07                  0
           3                           $99  2017-03                  0
...                                    ...      ...                ...
keller     3                          $160  2015-01                  0
诺斯莱克       3                           $58  2015-12                  0
           3                           $58  2015-12                  0
阿纳瓦克       3                           $29  2017-04                  0
           3                           $29  2017-04                  0

[18259 rows x 3 columns]

In [407]:
df_list = []
for region_beds in airbnb_df_new.index:
    counter = 1 
    region_bed_df = airbnb_df_new.loc[region_beds]
    region_bed_df.sort_values("Date")
    for date in region_bed_df.index:
        region_bed_df["Num_ABNB_Listings"] = counter
        counter +=1
    df_list.append(region_bed_df)


KeyboardInterrupt: 

In [ ]:
airbnb_df_newer = pd.concat(df_list)

In [398]:
airbnb_df_newer

,average_rate_per_night,Num_ABNB_Listings
Date,,
2009-07,$65,<generator object <genexpr> at 0x000001978DE0D...
2009-07,$65,<generator object <genexpr> at 0x000001978DE0D...
2013-04,$45,<generator object <genexpr> at 0x000001978DE0D...
2013-04,$45,<generator object <genexpr> at 0x000001978DE0D...
2013-07,$85,<generator object <genexpr> at 0x000001978DE0D...
...,...,...
2015-12,$58,<generator object <genexpr> at 0x000001978DE0D...
2017-04,$29,<generator object <genexpr> at 0x000001978DE0D...
2017-04,$29,<generator object <genexpr> at 0x000001978DE0D...


In [205]:
TX_nbhoods_df = pd.concat(df_list,join='inner')
counter = 0

In [206]:
TX_df = multi_index_data(TX_nbhoods_df)
TX_df

Empty DataFrame
Columns: []
Index: [(Abilene, 1, 2018-02), (Abilene, 1, 2018-03), (Abilene, 1, 2018-04), (Abilene, 1, 2018-05), (Abilene, 1, 2018-06), (Abilene, 1, 2018-07), (Abilene, 1, 2018-08), (Abilene, 1, 2018-09), (Abilene, 1, 2018-10), (Abilene, 1, 2018-11), (Abilene, 1, 2018-12), (Abilene, 1, 2019-01), (Abilene, 1, 2019-02), (Abilene, 1, 2019-03), (Abilene, 1, 2019-04), (Abilene, 1, 2019-05), (Abilene, 1, 2019-06), (Abilene, 1, 2019-07), (Abilene, 1, 2019-08), (Abilene, 1, 2019-09), (Abilene, 1, 2019-10), (Abilene, 1, 2019-11), (Abilene, 1, 2019-12), (Abilene, 2, 2017-02), (Abilene, 2, 2017-03), (Abilene, 2, 2017-04), (Abilene, 2, 2017-05), (Abilene, 2, 2017-06), (Abilene, 2, 2017-07), (Abilene, 2, 2017-08), (Abilene, 2, 2017-09), (Abilene, 2, 2017-10), (Abilene, 2, 2017-11), (Abilene, 2, 2017-12), (Abilene, 2, 2018-01), (Abilene, 2, 2018-02), (Abilene, 2, 2018-03), (Abilene, 2, 2018-04), (Abilene, 2, 2018-05), (Abilene, 2, 2018-06), (Abilene, 2, 2018-07), (Abilene, 2, 2018-08), (Abilene, 2, 2018-09), (Abilene, 2, 2018-10), (Abilene, 2, 2018-11), (Abilene, 2, 2018-12), (Abilene, 2, 2019-01), (Abilene, 2, 2019-02), (Abilene, 2, 2019-03), (Abilene, 2, 2019-04), (Abilene, 2, 2019-05), (Abilene, 2, 2019-06), (Abilene, 2, 2019-07), (Abilene, 2, 2019-08), (Abilene, 2, 2019-09), (Abilene, 2, 2019-10), (Abilene, 2, 2019-11), (Abilene, 2, 2019-12), (Abilene, 3, 2016-11), (Abilene, 3, 2016-12), (Abilene, 3, 2017-01), (Abilene, 3, 2017-02), (Abilene, 3, 2017-03), (Abilene, 3, 2017-04), (Abilene, 3, 2017-05), (Abilene, 3, 2017-06), (Abilene, 3, 2017-07), (Abilene, 3, 2017-08), (Abilene, 3, 2017-09), (Abilene, 3, 2017-10), (Abilene, 3, 2017-11), (Abilene, 3, 2017-12), (Abilene, 3, 2018-01), (Abilene, 3, 2018-02), (Abilene, 3, 2018-03), (Abilene, 3, 2018-04), (Abilene, 3, 2018-05), (Abilene, 3, 2018-06), (Abilene, 3, 2018-07), (Abilene, 3, 2018-08), (Abilene, 3, 2018-09), (Abilene, 3, 2018-10), (Abilene, 3, 2018-11), (Abilene, 3, 2018-12), (Abilene, 3, 2019-01), (Abilene, 3, 2019-02), (Abilene, 3, 2019-03), (Abilene, 3, 2019-04), (Abilene, 3, 2019-05), (Abilene, 3, 2019-06), (Abilene, 3, 2019-07), (Abilene, 3, 2019-08), (Abilene, 3, 2019-09), (Abilene, 3, 2019-10), (Abilene, 3, 2019-11), (Abilene, 3, 2019-12), (Abilene, 1, 2009-07), (Abilene, 1, 2009-07), (Abilene, 1, 2013-04), (Abilene, 1, 2013-04), ...]

[32342 rows x 0 columns]

In [145]:
housing_df= housing_df.melt(id_vars=["Region_ID", "Bedrooms"], 
        var_name="Date", 
        value_name="Median_Rent")
housing_df.dropna(inplace=True)
housing_df

KeyError: "The following 'id_vars' are not present in the DataFrame: ['Region_ID']"

In [97]:
housing_df

Median_Rent
Region_ID Bedrooms Date                
10        3        2010-03       1300.0
                   2010-04       1297.5
                   2010-05       1500.0
                   2010-06       1500.0
                   2010-07       1310.0
...                                 ...
288       4        2019-12       1950.0
292       4        2019-12       1912.5
294       4        2019-12       1800.0
298       4        2019-12       1547.5
309       4        2019-12       2275.0

[14083 rows x 1 columns]

In [99]:
housing_df.to_csv('../Data/median_rents_multi_indexed.csv')